# CWL Hands-on

CWL is a way to describe command-line tools and connect them together to create workflows. 

Because CWL is a specification and not a specific piece of software, tools and workflows described using CWL are portable across a variety of platforms that support the CWL standard.

CWL documents are written in YAML (and/or JSON). 

The example below shows a simple CWL “Hello World” workflow annotated with comments. Note that comments start with #:

In [1]:
cat quick-start.cwl

cwlVersion: v1.2

# What type of CWL process we have in this document.
class: CommandLineTool
# This CommandLineTool executes the rasterio "rio" command-line tool.
baseCommand: rio
# The arguments for this process
arguments: 
- bounds
# The inputs for this process.
inputs:
  geotiff:
    type: string
    # A default value that can be overridden, e.g. --message "Hola mundo"
    default: "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/TCI.tif"
    # Bind this message value as an argument to "echo".
    inputBinding:
      position: 1
outputs: []

requirements:
    DockerRequirement:
        dockerPull: localhost/rio:latest
    NetworkAccess:
        networkAccess: true


The example above is just a wrapper for the rasterio `rio bounds` command-line tool. 

Running the workflow above with the default input values will produce the same result as the command-line:


```bash
rio bounds "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/TCI.tif"
```

**Note**: In CWL, there is a distinction between a command-line tool and a workflow. But for the sake of simplicity, we are using the term “workflow” here.

## CWL Runner

`cwltool`` is an implementation of the CWL specification. 

It is also the CWL Reference Runner for the specification, and it is compliant with the latest version of the specification: v1.2. 

`cwltool` is installed using pip:

```
pip install cwltool
```

It is already install in our environment:

In [2]:
cwltool --help

usage: cwltool [-h] [--basedir BASEDIR] [--outdir OUTDIR] [--log-dir LOG_DIR]
               [--parallel]
               [--preserve-environment ENVVAR | --preserve-entire-environment]
               [--rm-container | --leave-container]
               [--cidfile-dir CIDFILE_DIR] [--cidfile-prefix CIDFILE_PREFIX]
               [--tmpdir-prefix TMPDIR_PREFIX]
               [--tmp-outdir-prefix TMP_OUTDIR_PREFIX | --cachedir CACHEDIR]
               [--rm-tmpdir | --leave-tmpdir]
               [--move-outputs | --leave-outputs | --copy-outputs]
               [--enable-pull | --disable-pull]
               [--rdf-serializer RDF_SERIALIZER] [--eval-timeout EVAL_TIMEOUT]
               [--provenance PROVENANCE] [--enable-user-provenance]
               [--disable-user-provenance] [--enable-host-provenance]
               [--disable-host-provenance] [--orcid ORCID]
               [--full-name CWL_FULL_NAME]
               [--print-rdf | --print-dot | --print-pre | --print-deps | --print-i

## Building the container

Create the Dockerfile:

In [ ]:
cat << 'EOF' > Dockerfile
ARG GDAL=ubuntu-small-3.6.4

FROM ghcr.io/osgeo/gdal:${GDAL} AS gdal

ARG PYTHON_VERSION=3.11

ENV LANG="C.UTF-8" LC_ALL="C.UTF-8"

RUN apt-get update -qq && \
    apt-get install -y software-properties-common && \
    add-apt-repository -y ppa:deadsnakes/ppa && \
    apt-get update -qq && \
    DEBIAN_FRONTEND=noninteractive apt-get install -y --no-install-recommends \
    g++ \
    gdb \
    make \
    python3-pip \
    jq \
    python${PYTHON_VERSION} && \
    rm -rf /var/lib/apt/lists/* && \
    pip install --no-cache-dir rasterio
EOF

Build the container:

In [ ]:
podman build -t localhost/rio -f Dockerfile .

Check the built container image:

In [ ]:
podman images

## Running the Quick Start workflow

The usage of the cwltool command-line executable is basically:

```
cwltool [OPTIONS] [CWL_DOCUMENT] [INPUTS_OBJECT]. 
```

You can run the `quickstart.cwl` workflow without specifying any option:



In [3]:
cwltool quick-start.cwl

INFO /opt/conda/bin/cwltool 3.1.20231114134824
INFO Resolved 'quick-start.cwl' to 'file:///workspace/cwl-guide-for-eo/notebooks/01%20-%20%20Quick%20start/quick-start.cwl'
INFO [job quick-start.cwl] /tmp/3iaabx0g$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/3iaabx0g,target=/yYqziS \
    --mount=type=bind,source=/tmp/m1m442i3,target=/tmp \
    --workdir=/yYqziS \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --cidfile=/tmp/j0813sbx/20240402112130-209968.cid \
    --env=TMPDIR=/tmp \
    --env=HOME=/yYqziS \
    localhost/rio:latest \
    rio \
    bounds \
    https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/TCI.tif
{"bbox": [-121.83433447992233, 39.635871469956584, -120.5195506800343, 40.64479964952879], "geometry": {"coordinates": [[[-121.83433447992233, 39.635871469956584], [-120.5195506800343, 39.635871469956584], [-120.5195506800343, 40.64479964952879], [-121.834334479

**Why is the process executed in a container?**

We have set an environment variable `CWLTOOL_OPTIONS=--podman` that instructs `cwltool` to run all workflows in containers using `podman`:

In [4]:
env | grep CWL

CWLTOOL_OPTIONS=--podman


You can override the default value of the input parameter `geotiff`, similar to how you would change the argument of the rasterio `rio bounds` command:

Running hello_world.cwl with cwltool passing an input parameter.

In [5]:
cwltool quick-start.cwl --geotiff "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B01.tif"

INFO /opt/conda/bin/cwltool 3.1.20231114134824
INFO Resolved 'quick-start.cwl' to 'file:///workspace/cwl-guide-for-eo/notebooks/01%20-%20%20Quick%20start/quick-start.cwl'
INFO [job quick-start.cwl] /tmp/puojp6ut$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/puojp6ut,target=/VsKBwf \
    --mount=type=bind,source=/tmp/pl_fz7w6,target=/tmp \
    --workdir=/VsKBwf \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --cidfile=/tmp/w4w5j748/20240402112137-172275.cid \
    --env=TMPDIR=/tmp \
    --env=HOME=/VsKBwf \
    localhost/rio:latest \
    rio \
    bounds \
    https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B01.tif
{"bbox": [-121.83433447992233, 39.635871469956584, -120.5195506800343, 40.64479964952879], "geometry": {"coordinates": [[[-121.83433447992233, 39.635871469956584], [-120.5195506800343, 39.635871469956584], [-120.5195506800343, 40.64479964952879], [-121.834334479

Another way of passing values to your workflow input parameters is via an Inputs Object. 

This is a file containing the input fields with their corresponding values. 

The Inputs Objects file can be written in JSON or YAML. For example:

In [6]:
cat inputs.yaml

geotiff: "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B02.tif"


You can use this Inputs Object file now to execute the workflow:

In [7]:
cwltool quick-start.cwl inputs.yaml

INFO /opt/conda/bin/cwltool 3.1.20231114134824
INFO Resolved 'quick-start.cwl' to 'file:///workspace/cwl-guide-for-eo/notebooks/01%20-%20%20Quick%20start/quick-start.cwl'
INFO [job quick-start.cwl] /tmp/1daywddp$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/1daywddp,target=/YSFuQU \
    --mount=type=bind,source=/tmp/r3_nomdz,target=/tmp \
    --workdir=/YSFuQU \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --cidfile=/tmp/oxkpetp2/20240402112144-279897.cid \
    --env=TMPDIR=/tmp \
    --env=HOME=/YSFuQU \
    localhost/rio:latest \
    rio \
    bounds \
    https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B02.tif
{"bbox": [-121.83433447992233, 39.635871469956584, -120.5195506800343, 40.64479964952879], "geometry": {"coordinates": [[[-121.83433447992233, 39.635871469956584], [-120.5195506800343, 39.635871469956584], [-120.5195506800343, 40.64479964952879], [-121.834334479